<a href="https://colab.research.google.com/github/arkeodev/XAI/blob/main/Layerwise_Relevance_Propagation_(LRP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Layerwise Relevance Propagation (LRP)

LRP is a technique used to explain the predictions of complex models by tracing the prediction back through the layers of the network to the input features, thereby providing a visual map or a set of influential features that led to the decision.

## Implementatation

## References

Layer-Wise Relevance Propagation: An Overview (Paper): https://iphome.hhi.de/samek/pdf/MonXAI19.pdf